<a href="https://colab.research.google.com/github/gadilshina-ve/SoftwareTools_DA/blob/main/LW_03/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime
import pandas as pd  # type: ignore
import random
import json


In [ ]:
# Функция для генерации CSV файла с данными о партнерах
def generate_csv_data():
    data_csv = {
        'ID': [i for i in range(1, 101)],
        'Сумма сделок': [random.randint(10000, 50000) for _ in range(100)],
        'Категория партнера': [random.choice(['Поставщик', 'Клиент', 'Партнер']) for _ in range(100)],
    }
    df_csv = pd.DataFrame(data_csv)
    df_csv.to_csv('partners_data.csv', index=False, encoding='utf-8')


# Функция для генерации Excel файла с данными о партнерах
def generate_excel_data():
    data_excel = {
        'ID': [i for i in range(51, 151)],
        'Количество сделок': [random.randint(1, 10) for _ in range(100)],
        'Категория партнера': [random.choice(['Поставщик', 'Клиент', 'Партнер']) for _ in range(100)],
    }
    df_excel = pd.DataFrame(data_excel)
    df_excel.to_excel('partners_transactions.xlsx', index=False)


# Функция для генерации JSON файла с данными о партнерах
def generate_json_data():
    data_json = [
        {'ID': i, 'Средняя сумма сделки': round(random.uniform(1000.0, 20000.0), 2), 'Категория партнера': random.choice(['Поставщик', 'Клиент', 'Партнер'])}
        for i in range(26, 126)
    ]
    with open('partners_average_deals.json', 'w', encoding='utf-8') as f:
        json.dump(data_json, f, ensure_ascii=False)


# Функция для обработки данных (очистка, удаление дубликатов и заполнение пропусков)
def clean_data():
    # Загрузка данных
    df_csv = pd.read_csv('partners_data.csv', encoding='utf-8')
    df_excel = pd.read_excel('partners_transactions.xlsx')
    with open('partners_average_deals.json', 'r', encoding='utf-8') as f:
        data_json = json.load(f)
    df_json = pd.DataFrame(data_json)

    # Объединение всех данных по 'ID'
    merged_df = pd.merge(df_csv, df_excel, on='ID', how='outer')
    merged_df = pd.merge(merged_df, df_json, on='ID', how='outer')

    # Очистка данных: удаление дубликатов и обработка пропусков
    df=merged_df.drop_duplicates(subset='ID', keep='first', inplace=True)
    df=merged_df.fillna({'Сумма сделок': 0, 'Количество сделок': 0, 'Средняя сумма сделки': 0}, inplace=True)

In [ ]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from airflow.operators.email_operator import EmailOperator
from datetime import datetime
import pandas as pd
import random
import json
import os


# Функция для генерации CSV файла с данными о партнерах
def generate_csv_data():
    data_csv = {
        'ID': [i for i in range(1, 101)],
        'Сумма сделок': [random.randint(10000, 50000) for _ in range(100)],
        'Категория партнера': [random.choice(['Поставщик', 'Клиент', 'Партнер']) for _ in range(100)],
    }
    df_csv = pd.DataFrame(data_csv)
    df_csv.to_csv('partners_data.csv', index=False, encoding='utf-8')

# Функция для генерации JSON файла с данными о партнерах
def generate_json_data():
    data_json = [
        {'ID': i, 'Средняя сумма сделки': round(random.uniform(1000.0, 20000.0), 2), 'Категория партнера': random.choice(['Поставщик', 'Клиент', 'Партнер'])}
        for i in range(26, 126)
    ]
    with open('partners_average_deals.json', 'w', encoding='utf-8') as f:
        json.dump(data_json, f, ensure_ascii=False)

# Функция для генерации Excel файла с данными о партнерах
def generate_excel_data():
    data_excel = {
        'ID': [i for i in range(51, 151)],
        'Количество сделок': [random.randint(1, 10) for _ in range(100)],
        'Категория партнера': [random.choice(['Поставщик', 'Клиент', 'Партнер']) for _ in range(100)],
    }
    df_excel = pd.DataFrame(data_excel)
    df_excel.to_excel('partners_transactions.xlsx', index=False)



# Функция для обработки данных (очистка, удаление дубликатов и заполнение пропусков)
def clean_data():
    # Загрузка данных
    df_csv = pd.read_csv('partners_data.csv', encoding='utf-8')
    df_excel = pd.read_excel('partners_transactions.xlsx')
    with open('partners_average_deals.json', 'r', encoding='utf-8') as f:
        data_json = json.load(f)
    df_json = pd.DataFrame(data_json)

    # Объединение всех данных по 'ID'
    merged_df = pd.merge(df_csv, df_json, on='ID', how='outer')
    merged_df = pd.merge(merged_df,df_excel, on='ID', how='outer')

    # Очистка данных: удаление дубликатов и обработка пропусков
    merged_df.drop_duplicates(subset='ID', keep='first', inplace=True)
    merged_df.fillna({'Сумма сделок': 0, 'Количество сделок': 0, 'Средняя сумма сделки': 0}, inplace=True)

    # Сохранение результата в Excel файл
    merged_df.to_excel('cleaned_partners_data.xlsx', index=False)


# Определение DAG
dag = DAG('partners_data_processing_dag', description='Генерация и очистка данных о партнерах',
          schedule_interval='@once', start_date=datetime(2023, 10, 1), catchup=False)

# Определение задач
task_generate_csv_data = PythonOperator(task_id='generate_csv_data', python_callable=generate_csv_data, dag=dag)
task_generate_json_data = PythonOperator(task_id='generate_json_data', python_callable=generate_json_data, dag=dag)
task_generate_excel_data = PythonOperator(task_id='generate_excel_data', python_callable=generate_excel_data, dag=dag)
task_clean_data = PythonOperator(task_id='clean_data', python_callable=clean_data, dag=dag)

# Установка зависимостей между задачами
task_generate_csv_data >> task_generate_json_data >> task_generate_excel_data >> task_clean_data
